https://neo4j.com/labs/genai-ecosystem/graphrag-python/

https://neo4j.com/docs/neo4j-graphrag-python/current/user_guide_rag.html#

In [5]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USERNAME = "neo4j"
NEO4J_PASSWORD = "anatomy-exodus-pigment-colombo-octopus-1216"

In [11]:
# Neo4j Driver
import neo4j

neo4j_driver = neo4j.GraphDatabase.driver(
    NEO4J_URI,
    auth=(NEO4J_USERNAME, NEO4J_PASSWORD)
)

# LLM and Embedding Model
from neo4j_graphrag.llm import OpenAILLM
from neo4j_graphrag.embeddings.openai import OpenAIEmbeddings

llm=OpenAILLM(
   model_name="gpt-4o-mini",
   model_params={
       "temperature": 0 # turning temperature down for more deterministic results
   }
)

embedder = OpenAIEmbeddings()

In [17]:
q = "Moamen Mostafa 가 한 질문들을 알려줘."

In [8]:
# Vector Retriever
from neo4j_graphrag.generation import GraphRAG, RagTemplate
from neo4j_graphrag.retrievers import VectorRetriever

vector_retriever = VectorRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    return_properties=["text"],
)

vector_rag = GraphRAG(llm=llm, retriever=vector_retriever, prompt_template=RagTemplate())

vector_rag.search(q, retriever_config={'top_k': 5})

Exception: No index with name text_embeddings found

In [19]:
# GraphRAG Vector Cypher Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import VectorCypherRetriever

graph_retriever = VectorCypherRetriever(
    neo4j_driver,
    index_name="text_embeddings",
    embedder=embedder,
    retrieval_query="""
//1) Go out 2-3 hops in the entity graph and get relationships
WITH node AS chunk
MATCH (chunk)<-[:FROM_CHUNK]-(entity)-[relList:!FROM_CHUNK]-{1,2}(nb)
UNWIND relList AS rel

//2) collect relationships and text chunks
WITH collect(DISTINCT chunk) AS chunks, collect(DISTINCT rel) AS rels

//3) format and return context
RETURN apoc.text.join([c in chunks | c.text], '\n') +
  apoc.text.join([r in rels |
  startNode(r).name+' - '+type(r)+' '+r.details+' -> '+endNode(r).name],
  '\n') AS info
"""
)

graph_rag = GraphRAG(llm=llm, retriever=graph_retriever, prompt_template=RagTemplate())

graph_rag.search(q, retriever_config={'top_k': 5})

Exception: No index with name text_embeddings found

In [22]:
# Text2Cypher Retriever
from neo4j_graphrag.generation import GraphRAG
from neo4j_graphrag.retrievers import Text2CypherRetriever

text2cypher_retriever = Text2CypherRetriever(
    driver=neo4j_driver,
    llm=llm
)

text2cypher_rag = GraphRAG(llm=llm, retriever=text2cypher_retriever, prompt_template=RagTemplate())

text2cypher_rag.search(q)

RagResultModel(answer='죄송하지만, Moamen Mostafa가 한 질문들에 대한 구체적인 정보는 제공할 수 없습니다. 다른 질문이나 도움이 필요하신 부분이 있다면 말씀해 주세요!', retriever_result=None)